In [1]:
from collections import OrderedDict
import geopandas
import libpysal
from libpysal import cg, examples
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
# import matplotlib_scalebar
# from matplotlib_scalebar.scalebar import ScaleBar
import numpy
# import ortools
# from ortools.linear_solver import pywraplp
# import seaborn
import shapely
from shapely.geometry import Point
import spaghetti
import sys

# n clients and n facilities
client_count, facility_count = 400, 14

# candidate facilites to site
p_facilities = 3

# maximum coverage meters
max_coverage = 1000.0

# minimum coverage meters
min_coverage = 800.0

random_seeds = {"client": 3006, "facility": 1520}

title = "Neighborhood X"

streets = geopandas.read_file(examples.get_path("streets.shp"))
streets.crs = "esri:102649"
streets = streets.to_crs("epsg:2762")
streets.head()

,ID,Length,geometry
0,1.0,244.116229,"LINESTRING (222008.039 267347.937, 222008.067 ..."
1,2.0,375.974828,"LINESTRING (222007.859 267549.106, 222008.039 ..."
2,3.0,400.353405,"LINESTRING (221421.335 267804.115, 221412.310 ..."
3,4.0,660.000000,"LINESTRING (220876.024 268352.613, 220804.856 ..."
4,5.0,660.000000,"LINESTRING (220803.334 268398.049, 220917.907 ..."


In [2]:
def get_buffer(in_data, buff=50):
    """ geopandas.GeoDataFrame should be in a meters projection.
    Parameters
    ----------
    in_data : geopandas.GeoDataFrame
        GeoDataFrame of a shapefile representing a road network.
    buff : int or float
        Desired buffer distance. Default is 50 (meters).
    Returns
    -------
    out_data : geopandas.GeoDataFrame
        Single polygon of the unioned street buffers.
    """
    b1 = in_data.buffer(buff)  # Buffer
    ub = b1.unary_union  # Buffer Union
    b2 = geopandas.GeoSeries(ub)
    out_data = geopandas.GeoDataFrame(b2, crs=in_data.crs, columns=["geometry"])
    return out_data

buff = 50
streets_buffer = get_buffer(streets, buff=buff)
streets_buffer

,geometry
0,"POLYGON ((221825.048 268652.947, 222026.216 26..."


In [3]:
def simulated_geo_points(in_data, needed=20, seed=0, to_file=None):
    """Generate synthetic spatial data points within an area.s
    Parameters
    ----------
    in_data : geopandas.GeoDataFrame
        A single polygon of the unioned street buffers.
    needed : int
        Number of points in the buffer. Default is 20.
    seed : int
        Seed for pseudo-random number generation. Default is 0.
    to_file : str
        File name for write out.
    Returns
    -------
    sim_pts : geopandas.GeoDataFrame
        Points within the buffer.
    """
    geoms = in_data.geometry
    area = tuple(in_data.total_bounds)
    simulated_points_list = []
    simulated_points_all = False
    numpy.random.seed(seed)
    while simulated_points_all == False:
        x = numpy.random.uniform(area[0], area[2], 1)
        y = numpy.random.uniform(area[1], area[3], 1)
        point = Point(x, y)
        if geoms.intersects(point)[0]:
            simulated_points_list.append(point)
        if len(simulated_points_list) == needed:
            simulated_points_all = True
    sim_pts = geopandas.GeoDataFrame(
        simulated_points_list, columns=["geometry"], crs=in_data.crs
    )
    if to_file:
        sim_pts.to_file(to_file + ".shp")
    return sim_pts

clients = simulated_geo_points(
    streets_buffer, needed=client_count, seed=random_seeds["client"]
)
facilities = simulated_geo_points(
    streets_buffer, needed=facility_count, seed=random_seeds["facility"]
)

In [4]:
clients["dv"] = ["x[%s]" % c for c in range(client_count)]
facilities["dv"] = ["y[%s]" % c for c in range(facility_count)]


In [5]:
numpy.random.seed(1991)
clients["weights"] = numpy.random.randint(1, 8, (client_count, 1))
clients.head()

,geometry,dv,weights
0,POINT (220622.824 267349.656),x[0],1
1,POINT (220804.074 268059.828),x[1],7
2,POINT (221871.690 268396.891),x[2],4
3,POINT (220716.905 267147.549),x[3],6
4,POINT (221331.363 267984.797),x[4],5


In [6]:
ntw = spaghetti.Network(in_data=streets)

In [7]:
ntw.snapobservations(clients, "clients", attribute=True)
clients_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="clients", snapped=True
)
ntw.snapobservations(facilities, "facilities", attribute=True)
facilities_snapped = spaghetti.element_as_gdf(
    ntw, pp_name="facilities", snapped=True
)

In [8]:
cost_matrix = ntw.allneighbordistances(
    sourcepattern=ntw.pointpatterns["clients"],
    destpattern=ntw.pointpatterns["facilities"],
)

In [9]:
import pulp
r_fac = range(cost_matrix.shape[1])
r_cli = range(cost_matrix.shape[0])

s = max_coverage # meters
# binary coverage matrix from cij
aij = numpy.zeros(cost_matrix.shape)
aij[cost_matrix <= s] = 1.0

fac_vars = [pulp.LpVariable("y[%i]" % (j), lowBound=0, upBound=1, cat="Integer") for j in r_fac]
print(pulp.lpSum(fac_vars))
# objective function
model = pulp.LpProblem("LSCP", pulp.LpMinimize)
model += pulp.lpSum(fac_vars), "objective function"
for i in r_cli:
    model += pulp.lpSum([aij[i][j] * fac_vars[j] for j in r_fac]) >= 1
model.writeLP("lscp.lp")
model.solve()

for v in model.variables():
    print(v.name, "=", v.varValue)

y_0_ + y_10_ + y_11_ + y_12_ + y_13_ + y_1_ + y_2_ + y_3_ + y_4_ + y_5_ + y_6_ + y_7_ + y_8_ + y_9_
y_0_ = 0
y_10_ = 1
y_11_ = 0
y_12_ = 1
y_13_ = 1
y_1_ = 0
y_2_ = 0
y_3_ = 0
y_4_ = 1
y_5_ = 0
y_6_ = 0
y_7_ = 0
y_8_ = 0
y_9_ = 0


In [10]:
import pulp

r_fac = range(cost_matrix.shape[1])
r_cli = range(cost_matrix.shape[0])

s = max_coverage # meters
# binary coverage matrix from cij
aij = numpy.zeros(cost_matrix.shape)
aij[cost_matrix <= s] = 1.0
ai = clients['weights']
ai_sum = ai.sum()
ai = ai.values.reshape(cost_matrix.shape[0], 1)

fac_vars = [pulp.LpVariable(f"x[{j}]", lowBound=0, upBound=1, cat="Integer") for j in r_fac]
dem_vars = [pulp.LpVariable(f"y[{i}]", lowBound=0, upBound=1, cat="Integer") for i in r_cli]

model = pulp.LpProblem("MCLP", pulp.LpMaximize)
model += pulp.lpSum([ai.flatten()[i] * dem_vars[i] for i in r_cli]), "objective function"
for i in r_cli:
    model += pulp.lpSum([aij[i][j] * fac_vars[j] for j in r_fac]) >= dem_vars[i]

model += pulp.lpSum(fac_vars) == p_facilities
model.solve()

# for v in model.variables():
#     print(v.name, "=", v.varValue)

1

In [11]:
import pulp

r_fac = range(cost_matrix.shape[1])
r_cli = range(cost_matrix.shape[0])

s = max_coverage # meters

# binary coverage matrix from cij
aij = numpy.zeros(cost_matrix.shape)
aij[cost_matrix <= s] = 1.0
ai = clients['weights']
ai_sum = ai.sum()
ai = ai.values.reshape(cost_matrix.shape[0], 1)

# weighted demand
try:
    sij = ai * cost_matrix
except ValueError:
    ai = ai.values.reshape(cost_matrix.shape[0], 1)
    sij = ai * cost_matrix

zij = [[pulp.LpVariable(f"z[{i}_{j}]", lowBound=0, upBound=1, cat="Integer") for j in r_fac] for i in r_cli]
fac_vars = [pulp.LpVariable(f"y[{j}]", lowBound=0, upBound=1, cat="Integer") for j in r_fac]
model = pulp.LpProblem("P-median", pulp.LpMinimize)
model += pulp.lpSum([sij[i][j] * zij[i][j] for i in r_cli for j in r_fac]), "objective function"
for i in r_cli:
    model += pulp.lpSum([zij[i][j] for j in r_fac])==1
model += pulp.lpSum([fac_vars[j] for j in r_fac])==p_facilities

for i in r_cli:
    for j in r_fac:
        model += fac_vars[j] - zij[i][j] >= 0

model.solve()

1

In [12]:
import pulp

r_fac = range(cost_matrix.shape[1])
r_cli = range(cost_matrix.shape[0])

s = max_coverage # meters

# binary coverage matrix from cij
aij = numpy.zeros(cost_matrix.shape)
aij[cost_matrix <= s] = 1.0
ai = clients['weights']
ai_sum = ai.sum()
ai = ai.values.reshape(cost_matrix.shape[0], 1)

# weighted demand
try:
    sij = ai * cost_matrix
except ValueError:
    ai = ai.values.reshape(cost_matrix.shape[0], 1)
    sij = ai * cost_matrix

model = pulp.LpProblem('P-Center', pulp.LpMinimize)

w = pulp.LpVariable('W', lowBound=0, cat='Continuous')
zij = [[pulp.LpVariable(f"z[{i}_{j}]", lowBound=0, upBound=1, cat="Integer") for j in r_fac] for i in r_cli]
fac_vars = [pulp.LpVariable(f"y[{j}]", lowBound=0, upBound=1, cat="Integer") for j in r_fac]

model += w, 'objective function'
for i in r_cli:
    model += pulp.lpSum([zij[i][j] for j in r_fac])==1
model += pulp.lpSum([fac_vars[j] for j in r_fac])==p_facilities
for i in r_cli:
    for j in r_fac:
        model += fac_vars[j] - zij[i][j] >= 0

for i in r_cli:
    model += pulp.lpSum([zij[i][j]*cost_matrix[i][j] for j in r_fac]) <= w

model.solve()

1